<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/FinalModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

#Resnet50

##Original224

In [3]:

import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/FineTune/Resnet50/Vector_Resnet50_224.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/Vector_Resnet50_224'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/Vector_Resnet50_224


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes, similarity_scores

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/Vector_Resnet50_224'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บภาพที่จะทดสอบ
test_image_folder = '/content/drive/MyDrive/drug/Test_20_1000_Corp'

# ฟังก์ชันสำหรับดึงชื่อคลาสที่แท้จริงจากชื่อโฟลเดอร์ย่อย
def get_class_from_filename(image_path):
    return os.path.basename(os.path.dirname(image_path))

# เก็บข้อมูล similarity ของภาพแต่ละภาพในตัวแปร
image_similarity_results = {}

true_classes = {}
for root, dirs, files in os.walk(test_image_folder):
    for image_file in files:
        if image_file.endswith(('jpg', 'png')):
            image_path = os.path.join(root, image_file)
            true_class = get_class_from_filename(image_path)
            true_classes[image_file] = true_class

# ประมวลผลภาพทั้งหมดในโฟลเดอร์ย่อย
for root, dirs, files in os.walk(test_image_folder):
    for image_file in files:
        if image_file.endswith(('jpg', 'png')):
            image_path = os.path.join(root, image_file)

            # ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
            similar_classes, similarity_scores = find_most_similar_classes(image_path, all_embeddings)

            # เก็บผลลัพธ์ในตัวแปร
            image_similarity_results[image_file] = {
                'similar_classes': similar_classes,
                'similarity_scores': similarity_scores
            }

            # แสดงผลลัพธ์
            print(f"Results for image: {image_file}")
            for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
                print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

            print("\n" + "="*40 + "\n")

# สร้างกราฟแสดงผลการทำนาย
def plot_prediction_results(image_similarity_results, true_classes):
    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        similar_classes = result['similar_classes']
        is_correct = [1 if class_name == true_class else 0 for class_name, _ in similar_classes]
        labels = [class_name for class_name, _ in similar_classes]
        scores = [score for _, score in similar_classes]

# แสดงกราฟผลการทำนาย
plot_prediction_results(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_heatmap(image_similarity_results, true_classes):
    # เก็บชื่อคลาสทั้งหมดที่มี และชื่อโฟลเดอร์ย่อยทั้งหมด
    unique_classes = set()
    subfolders = set()

    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        labels = [class_name for class_name, _ in similar_classes]
        unique_classes.update(labels)

        # ดึงชื่อโฟลเดอร์ย่อยที่ไฟล์อยู่ (ซึ่งแทนคลาสที่ถูกต้อง)
        true_class = true_classes.get(image_file, None)
        if true_class:
            subfolders.add(true_class)

    unique_classes = sorted(unique_classes)  # เรียงชื่อคลาสตามตัวอักษร
    subfolders = sorted(subfolders)  # เรียงชื่อโฟลเดอร์ย่อยตามตัวอักษร

    # สร้าง dictionary เพื่อเก็บผลรวมการทำนายในแต่ละโฟลเดอร์ย่อย
    folder_counts = {folder: {class_name: 0 for class_name in unique_classes} for folder in subfolders}

    for image_file, result in image_similarity_results.items():
        # หาคลาสที่มีคะแนนสูงสุด
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]  # คลาสที่ทำนายได้

        # ดึงชื่อโฟลเดอร์ย่อยที่ไฟล์อยู่ (ซึ่งแทนคลาสที่ถูกต้อง)
        true_class = true_classes.get(image_file, None)
        if true_class:
            # เพิ่มการนับในคลาสที่ทำนายได้ในโฟลเดอร์นั้น ๆ
            folder_counts[true_class][top_class] += 1

    # แปลงเป็น DataFrame
    counts_df = pd.DataFrame.from_dict(folder_counts, orient='index').fillna(0)

    # วาด Heatmap
    plt.figure(figsize=(15, 10))
    sns.heatmap(counts_df, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, cbar_kws={'label': 'Number of Predictions'})
    plt.title('Heatmap of Predicted Classes by True Class (Subfolder)')
    plt.xlabel('Predicted Class Names')
    plt.ylabel('True Class (Subfolder Names)')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)  # ทำให้ชื่อโฟลเดอร์แสดงในแนวนอน
    plt.tight_layout()
    plt.show()

# แสดง Heatmap สำหรับจำนวนการทำนายของแต่ละโฟลเดอร์ย่อย (ที่แทนคลาสที่ถูกต้อง) ว่าถูกทำนายเป็นคลาสไหนบ้าง
plot_heatmap(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_heatmap(image_similarity_results, true_classes):
    unique_classes = set()
    subfolders = set()

    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        labels = [class_name for class_name, _ in similar_classes]
        unique_classes.update(labels)

        true_class = true_classes.get(image_file, None)
        if true_class:
            subfolders.add(true_class)

    unique_classes = sorted(unique_classes)
    subfolders = sorted(subfolders)

    # สร้าง dictionary เพื่อเก็บผลรวมการทำนายในแต่ละโฟลเดอร์ย่อย
    folder_counts = {folder: {class_name: {'correct': 0, 'incorrect': 0} for class_name in unique_classes} for folder in subfolders}

    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]

        true_class = true_classes.get(image_file, None)
        if true_class:
            if top_class == true_class:
                folder_counts[true_class][top_class]['correct'] += 1
            else:
                folder_counts[true_class][top_class]['incorrect'] += 1

    # แปลงเป็น DataFrame สำหรับการทำนายถูกและผิด
    correct_counts = pd.DataFrame.from_dict(
        {folder: {class_name: counts['correct'] for class_name, counts in class_counts.items()} for folder, class_counts in folder_counts.items()},
        orient='index'
    ).fillna(0)

    incorrect_counts = pd.DataFrame.from_dict(
        {folder: {class_name: counts['incorrect'] for class_name, counts in class_counts.items()} for folder, class_counts in folder_counts.items()},
        orient='index'
    ).fillna(0)

    # วาด Heatmap สำหรับการทำนายถูก
    plt.figure(figsize=(15, 10))
    sns.heatmap(correct_counts, annot=True, fmt='.0f', cmap='Greens', linewidths=.5, cbar_kws={'label': 'Number of Correct Predictions'})
    plt.title('Heatmap of Correct Predictions by True Class (Subfolder)')
    plt.xlabel('Predicted Class Names')
    plt.ylabel('True Class (Subfolder Names)')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

    # วาด Heatmap สำหรับการทำนายผิด
    plt.figure(figsize=(15, 10))
    sns.heatmap(incorrect_counts, annot=True, fmt='.0f', cmap='Reds', linewidths=.5, cbar_kws={'label': 'Number of Incorrect Predictions'})
    plt.title('Heatmap of Incorrect Predictions by True Class (Subfolder)')
    plt.xlabel('Predicted Class Names')
    plt.ylabel('True Class (Subfolder Names)')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

# แสดง Heatmap สำหรับจำนวนการทำนายที่ถูกและผิด
plot_heatmap(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot_total_correct_incorrect_counts(image_similarity_results, true_classes):
    # สร้างตัวแปรเพื่อเก็บจำนวนการทำนายที่ถูกต้องและผิดพลาดทั้งหมด
    total_correct = 0
    total_incorrect = 0

    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]

        true_class = true_classes.get(image_file, None)
        if true_class:
            if top_class == true_class:
                total_correct += 1
            else:
                total_incorrect += 1

    # คำนวณ accuracy
    total_predictions = total_correct + total_incorrect
    accuracy = total_correct / total_predictions if total_predictions > 0 else 0

    # สร้าง DataFrame สำหรับจำนวนการทำนายที่ถูกต้องและผิดพลาดทั้งหมด
    df_counts = pd.DataFrame({
        'Prediction Type': ['Correct', 'Incorrect'],
        'Count': [total_correct, total_incorrect]
    })

    # กำหนดสีสำหรับแต่ละประเภทการทำนาย
    palette = {'Correct': 'royalblue', 'Incorrect': 'tomato'}

    # สร้างกราฟแท่งแสดงจำนวนการทำนายที่ถูกต้องและผิดพลาดทั้งหมด
    plt.figure(figsize=(8, 6))
    barplot = sns.barplot(data=df_counts, x='Prediction Type', y='Count', palette=palette)

    # เพิ่มตัวเลขบนแท่ง
    for p in barplot.patches:
        barplot.annotate(f'{int(p.get_height())}',
                         (p.get_x() + p.get_width() / 2., p.get_height()),
                         ha='center', va='center',
                         xytext=(0, 9), textcoords='offset points',
                         fontsize=12, color='black')

    plt.title(f'Total Number of Correct and Incorrect Predictions\nAccuracy: {accuracy:.2%}')
    plt.xlabel('Prediction Type')
    plt.ylabel('Number of Predictions')
    plt.tight_layout()
    plt.show()

    print(f'Accuracy: {accuracy:.2%}')

# แสดงกราฟจำนวนการทำนายที่ถูกต้องและผิดพลาดทั้งหมด
plot_total_correct_incorrect_counts(image_similarity_results, true_classes)

#VITModel

##Fine Tune224

In [3]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/FineTune/VITModel/model_VITModel_224.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/model_VITModel_224'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/model_VITModel_224


In [4]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/FineTune/VITModel/Vector_VITModel_FT224.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/Vector_VITModel_FT224'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/Vector_VITModel_FT224


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import matplotlib.pyplot as plt

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('/content/model_VITModel_224')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes, similarity_scores

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/Vector_VITModel_FT224'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บภาพที่จะทดสอบ
test_image_folder = '/content/drive/MyDrive/drug/Test_20_1000_Corp'

# ฟังก์ชันสำหรับดึงชื่อคลาสที่แท้จริงจากชื่อโฟลเดอร์ย่อย
def get_class_from_directory(image_path):
    return os.path.basename(os.path.dirname(image_path))

# เก็บข้อมูล similarity ของภาพแต่ละภาพในตัวแปร
image_similarity_results = {}

# เก็บชื่อคลาสที่แท้จริงจากชื่อไฟล์
true_classes = {}
for root, dirs, files in os.walk(test_image_folder):
    for image_file in files:
        if image_file.endswith(('jpg', 'png')):
            image_path = os.path.join(root, image_file)
            true_class = get_class_from_directory(image_path)
            true_classes[image_file] = true_class

# ประมวลผลภาพทั้งหมดในโฟลเดอร์
for root, dirs, files in os.walk(test_image_folder):
    for image_file in files:
        if image_file.endswith(('jpg', 'png')):
            image_path = os.path.join(root, image_file)

            # ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
            similar_classes, similarity_scores = find_most_similar_classes(image_path, all_embeddings)

            # เก็บผลลัพธ์ในตัวแปร
            image_similarity_results[image_file] = {
                'similar_classes': similar_classes,
                'similarity_scores': similarity_scores
            }

            # แสดงผลลัพธ์
            print(f"Results for image: {image_file}")
            for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
                print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

            print("\n" + "="*40 + "\n")

# สร้างกราฟแสดงผลการทำนาย
def plot_prediction_results(image_similarity_results, true_classes):
    for image_file, result in image_similarity_results.items():
        true_class = true_classes.get(image_file, None)
        if not true_class:
            continue

        similar_classes = result['similar_classes']
        is_correct = [1 if class_name == true_class else 0 for class_name, _ in similar_classes]
        labels = [class_name for class_name, _ in similar_classes]
        scores = [score for _, score in similar_classes]

# แสดงกราฟผลการทำนาย
plot_prediction_results(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_heatmap(image_similarity_results, true_classes):
    # เก็บชื่อคลาสทั้งหมดที่มี และชื่อโฟลเดอร์ย่อยทั้งหมด
    unique_classes = set()
    subfolders = set()

    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        labels = [class_name for class_name, _ in similar_classes]
        unique_classes.update(labels)

        # ดึงชื่อโฟลเดอร์ย่อยที่ไฟล์อยู่ (ซึ่งแทนคลาสที่ถูกต้อง)
        true_class = true_classes.get(image_file, None)
        if true_class:
            subfolders.add(true_class)

    unique_classes = sorted(unique_classes)  # เรียงชื่อคลาสตามตัวอักษร
    subfolders = sorted(subfolders)  # เรียงชื่อโฟลเดอร์ย่อยตามตัวอักษร

    # สร้าง dictionary เพื่อเก็บผลรวมการทำนายในแต่ละโฟลเดอร์ย่อย
    folder_counts = {folder: {class_name: 0 for class_name in unique_classes} for folder in subfolders}

    for image_file, result in image_similarity_results.items():
        # หาคลาสที่มีคะแนนสูงสุด
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]  # คลาสที่ทำนายได้

        # ดึงชื่อโฟลเดอร์ย่อยที่ไฟล์อยู่ (ซึ่งแทนคลาสที่ถูกต้อง)
        true_class = true_classes.get(image_file, None)
        if true_class:
            # เพิ่มการนับในคลาสที่ทำนายได้ในโฟลเดอร์นั้น ๆ
            folder_counts[true_class][top_class] += 1

    # แปลงเป็น DataFrame
    counts_df = pd.DataFrame.from_dict(folder_counts, orient='index').fillna(0)

    # วาด Heatmap
    plt.figure(figsize=(15, 10))
    sns.heatmap(counts_df, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, cbar_kws={'label': 'Number of Predictions'})
    plt.title('Heatmap of Predicted Classes by True Class (Subfolder)')
    plt.xlabel('Predicted Class Names')
    plt.ylabel('True Class (Subfolder Names)')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)  # ทำให้ชื่อโฟลเดอร์แสดงในแนวนอน
    plt.tight_layout()
    plt.show()

# แสดง Heatmap สำหรับจำนวนการทำนายของแต่ละโฟลเดอร์ย่อย (ที่แทนคลาสที่ถูกต้อง) ว่าถูกทำนายเป็นคลาสไหนบ้าง
plot_heatmap(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_heatmap(image_similarity_results, true_classes):
    unique_classes = set()
    subfolders = set()

    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        labels = [class_name for class_name, _ in similar_classes]
        unique_classes.update(labels)

        true_class = true_classes.get(image_file, None)
        if true_class:
            subfolders.add(true_class)

    unique_classes = sorted(unique_classes)
    subfolders = sorted(subfolders)

    # สร้าง dictionary เพื่อเก็บผลรวมการทำนายในแต่ละโฟลเดอร์ย่อย
    folder_counts = {folder: {class_name: {'correct': 0, 'incorrect': 0} for class_name in unique_classes} for folder in subfolders}

    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]

        true_class = true_classes.get(image_file, None)
        if true_class:
            if top_class == true_class:
                folder_counts[true_class][top_class]['correct'] += 1
            else:
                folder_counts[true_class][top_class]['incorrect'] += 1

    # แปลงเป็น DataFrame สำหรับการทำนายถูกและผิด
    correct_counts = pd.DataFrame.from_dict(
        {folder: {class_name: counts['correct'] for class_name, counts in class_counts.items()} for folder, class_counts in folder_counts.items()},
        orient='index'
    ).fillna(0)

    incorrect_counts = pd.DataFrame.from_dict(
        {folder: {class_name: counts['incorrect'] for class_name, counts in class_counts.items()} for folder, class_counts in folder_counts.items()},
        orient='index'
    ).fillna(0)

    # วาด Heatmap สำหรับการทำนายถูก
    plt.figure(figsize=(15, 10))
    sns.heatmap(correct_counts, annot=True, fmt='.0f', cmap='Greens', linewidths=.5, cbar_kws={'label': 'Number of Correct Predictions'})
    plt.title('Heatmap of Correct Predictions by True Class (Subfolder)')
    plt.xlabel('Predicted Class Names')
    plt.ylabel('True Class (Subfolder Names)')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

    # วาด Heatmap สำหรับการทำนายผิด
    plt.figure(figsize=(15, 10))
    sns.heatmap(incorrect_counts, annot=True, fmt='.0f', cmap='Reds', linewidths=.5, cbar_kws={'label': 'Number of Incorrect Predictions'})
    plt.title('Heatmap of Incorrect Predictions by True Class (Subfolder)')
    plt.xlabel('Predicted Class Names')
    plt.ylabel('True Class (Subfolder Names)')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

# แสดง Heatmap สำหรับจำนวนการทำนายที่ถูกและผิด
plot_heatmap(image_similarity_results, true_classes)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot_total_correct_incorrect_counts(image_similarity_results, true_classes):
    # สร้างตัวแปรเพื่อเก็บจำนวนการทำนายที่ถูกต้องและผิดพลาดทั้งหมด
    total_correct = 0
    total_incorrect = 0

    for image_file, result in image_similarity_results.items():
        similar_classes = result['similar_classes']
        top_class = similar_classes[0][0]

        true_class = true_classes.get(image_file, None)
        if true_class:
            if top_class == true_class:
                total_correct += 1
            else:
                total_incorrect += 1

    # คำนวณ accuracy
    total_predictions = total_correct + total_incorrect
    accuracy = total_correct / total_predictions if total_predictions > 0 else 0

    # สร้าง DataFrame สำหรับจำนวนการทำนายที่ถูกต้องและผิดพลาดทั้งหมด
    df_counts = pd.DataFrame({
        'Prediction Type': ['Correct', 'Incorrect'],
        'Count': [total_correct, total_incorrect]
    })

    # กำหนดสีสำหรับแต่ละประเภทการทำนาย
    palette = {'Correct': 'royalblue', 'Incorrect': 'tomato'}

    # สร้างกราฟแท่งแสดงจำนวนการทำนายที่ถูกต้องและผิดพลาดทั้งหมด
    plt.figure(figsize=(8, 6))
    barplot = sns.barplot(data=df_counts, x='Prediction Type', y='Count', palette=palette)

    # เพิ่มตัวเลขบนแท่ง
    for p in barplot.patches:
        barplot.annotate(f'{int(p.get_height())}',
                         (p.get_x() + p.get_width() / 2., p.get_height()),
                         ha='center', va='center',
                         xytext=(0, 9), textcoords='offset points',
                         fontsize=12, color='black')

    plt.title(f'Total Number of Correct and Incorrect Predictions\nAccuracy: {accuracy:.2%}')
    plt.xlabel('Prediction Type')
    plt.ylabel('Number of Predictions')
    plt.tight_layout()
    plt.show()

    print(f'Accuracy: {accuracy:.2%}')

# แสดงกราฟจำนวนการทำนายที่ถูกต้องและผิดพลาดทั้งหมด
plot_total_correct_incorrect_counts(image_similarity_results, true_classes)